<hr style="height:2px;border:none"/>
<h1 align='center'> INF-280 Estadística Computacional II-2022 </h1>

<H3 align='center'> Tema 8 - Test de Hipótesis  </H3>

\


Nombre del equipo: 

Integrantes:


*   Nombre Integrante 1 - ROL - [video](https://youtu.be/A6O67uLJfcQ)
*   Nombre Integrante 2 - ROL - [video](https://youtu.be/A6O67uLJfcQ)
*   Nombre Integrante 3 - ROL - [video](https://youtu.be/A6O67uLJfcQ)


*Recuerden que si su equipo se llama "El ritmo no perdona", entonces el nombre del archivo a subir debe ser T8_El_ritmo_no_perdona.ipynb*


### **Formalidades**  
* Es posible utilizar apuntes, libros, papers, ejemplos y cualquier otro material que desee de internet, pero debe ser usted quien desarrolle los ejercicios y comente los resultados en el Notebook que entregará. 
* Se permite el uso de las librerías: pandas, numpy, matplotlib y seaborn. Si desea utilizar otra librería debe indicar cómo instalarla.
* Se debe trabajar en grupos de **dos o tres personas**, las cuales deberán estar inscritas a través de __[Aula](https://aula.usm.cl/)__.
* Además del Notebook, deberán entregar un video explicando lo que hizo, paso a paso, como en los ejemplos entregados en AULA. Este vídeo es **individual**, es decir, cada integrante debe hacerlo por separado (debiendo utilizar el mismo Jupyter Notebook). El video puede ser subido a una plataforma externa como Youtube, Google Drive, ó Dropbox. El **link a compartir** (*visible para cualquier usuario con el link*) debe ir dentro del Notebook.
* La nota de esta actividad es grupal.


<hr style="height:2px;border:none"/>

### **Entrega** 
* **Formato**: Se debe realizar una **única** entrega por grupo (un representate del grupo) a través de  __[**Aula**](https://aula.usm.cl/)__.
* **Archivo**: El nombre del archivo a subir debe ser **T8_Nombre_Del_Equipo.ipynb**, por ejemplo, si su equipo se llama "El ritmo no perdona", entonces el nombre del archivo debe ser **T8_El_ritmo_no_perdona.ipynb**
* **Archivo**: Sus respuestas deben ser entregadas en forma de Jupyter Notebook, incluyendo **todos links** a los vídeos explicativos, uno por cada integrante.
    * **Vídeo**: Debe grabar la pantalla (Jupyter Notebok) donde se realiza la explicación de la actividad, con una duración máxima de 25 minutos.
* **Fecha límite**: Martes 6 de Diciembre a las 23:59

<hr style="height:2px;border:none"/>

### **Descuentos**

* Se descontarán 5 puntos si el nombre del archivo subido no sigue el formato señalado.
* Se descontarán 10 puntos por cada hora de atraso en la entrega.
* Todos los gráficos que realicen deben contener los nombres de las variables en los ejes. Se descontarán 3 puntos por cada nombre faltante.

<hr style="height:2px;border:none"/>

<table align='left'>
<tr>
<td><img src='https://comunicacion.uaa.mx/revista/wp-content/uploads/2019/01/discurso-620x350.png' width='650'/> </td>
</tr>
</table>

## Contexto

La Universidad de Harvard ha recopilado una serie de discursos para medir el nivel de populismo en los discursos de 241 jefes ejecutivos (presidentes y primeros ministros) de 74 países de todos los continentes. El conjunto de datos cubre 324 mandatos gubernamentales e incluye 1240 discursos, en su mayoría entre 2000 y 2022. 

El dataset `gpd_v2_20220427.csv` está compuesto por los siguientes atriibutos:

1.   `merging_variable`: name of the .txt file in the folder with Speeches with the speech that matches that coder’s score.
2.   `Country`
3.   `Leader`: name of president or prime minister.
4.   `lr`: categorical indicator with -1 = left; 0 = center; 1 = right.
5.   `president`: binary. 1 if the leader is a president, 0 if a prime minister.
6.   `term`: leader term in office, ordinal.
8.   `startofterm`: date of start of respective term
9.   `yearbegin`: year of beginning of term.
10.   `endofterm`: date of end of respective term.
11.   `yearend`: year of ending of respective term.
12.   `speechtype`: type of speech: international, campaign, ribbon-cutting, famous.
13.   `speechnum`: number of that speech for that leader-term.
14.   `codernum`: coder identifier for that speech-leader.
15.   `rubricgrade`: grade of that speech by that coder on the 0-2 populism scale.
16.   `averagerubric`: average populism grade for that speech. Arithmetic average across all coders for that speech.
17.    `totalaverage`: average populism grade for that leader-term. Arithmetic mean for all speeches by all coders for that leader-term
18.    `wb_region`: World Bank global region classification.

# Librerías

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns; sns.set()
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")


# Datos

In [ ]:
data = pd.read_csv("gpd_v2_20220427.csv")

# Preguntas

---

**Importante** para las siguientes preguntas:
* Indique explicitamente **los supuestos** que consideraron.
* Indique explicitamente **$H_0$** y **$H_1$**.
* Indique explicitamente el valor del **Estadístico de Contraste**.
* Indique explicitamente la **Región Crítica**.
* Indique explicitamente si **se acepta $H_0$** o **se rechaza $H_0$**.

---

1. Considerando solo los discursos realizados en  `region=Latin America & Caribbean` y `region=Western Europe` desde el año 2000 en adelante y solamente aquellos realizados en campaña (`speechtype = Campaign`). 

> a) Diseñe un contraste que permita evaluar la hipótesis de que "*los discursos realizados por presidentes de izquierda (`lr = -1`) tienen un populismo (`rubricgrade`) promedio **distinto** en ambas regiones*". Diseñe su contraste de manera que la probabilidad de aceptar incorrectamente esta conjetura sea menor al 5%. **(10 pts)**

> b) Diseñe un contraste que permita evaluar la hipótesis de que "*los discursos realizados por presidentes de centro (`lr = 0`) tienen un populismo (`rubricgrade`) promedio **distinto** en ambas regiones*". Diseñe un contraste de manera que la probabilidad de aceptar incorrectamente esta conjetura sea menor al 5%. **(10 pts)**

> c) Diseñe un contraste que permita evaluar la hipótesis de que "*los discursos realizados por presidentes de derecha (`lr = 1`) tienen un populismo (`rubricgrade`) promedio **distinto** en ambas regiones*". Diseñe un contraste de manera que la probabilidad de aceptar incorrectamente esta conjetura sea menor al 5%. **(10 pts)**

2. Considerando solo los discursos realizados en Chile desde el año 2000 en adelante. Diseñe un contraste que permita evaluar la hipótesis de que "*los discursos realizados por presidentes de izquierda (`lr = -1`) son **más populistas** (utilice `rubricgrade`) que los discursos realizados por presidentes de derecha (`lr = -1`)*". Diseñe un contraste de manera que la probabilidad de aceptar incorrectamente esta conjetura sea menor al 5%.  **(10 pts)**

3. En Chile, Michelle Bachelet y Sebastián Piñera han sido dos veces presidente de la república. Por esto, podría resultar interesante determinar si sus niveles de populismo (utilice `rubricgrade`) en sus discursos cambiaron en algo durante sus dos mandatos. 

> a) Ejecute un contraste donde $H_0$ corresponda a que "*los discursos de "Bachelet 1" son **igual o más populistas** que los discursos de "Bachelet 2"* ". Considere $α =  5\%$. **(15 pts)**

> b) Ejecute un contraste donde $H_0$ corresponda a que "*los discursos de "Piñera 1" son **igual o más populistas** que los discursos de "Piñera 2"* ". Considere $α =  5\%$. **(15 pts)**

4. En los últimos 20 años Argentina ha aumentado su nivel de pobreza, donde más del 25% de la población del país se encuentra en esta situación.

> a) Diseñe un contraste que permita evaluar la hipótesis de que "*el populismo (utilice `rubricgrade`) promedio en los discursos de presidentes argentinos es mayor a 0.5*". Diseñe un contraste de manera que la probabilidad de aceptar incorrectamente esta conjetura sea menor al 5%. **(10 pts)**

> b) Diseñe un contraste que permita evaluar la hipótesis de que "*los discursos realizados por presidentes argentinos son más populistas que los discursos realizados por el resto de países de `region=Latin America & Caribbean`*". Diseñe un contraste de manera que la probabilidad de aceptar incorrectamente esta conjetura sea menor al 5%  **(10 pts)**

**Debido a los problemas con el dataset de la pregunta 5), se ha decidido eliminar dicha pregunta. Por esto, se otorgará un puntaje base de 20 pts.**

# Ejemplo Test de Hipótesis:


En un test de hipótesis, la hipótesis nula ($H_0$) corresponde al *status quo*, es decir, a lo que se "cree que es lo normal". En cambio, la hipótesis alternativa es lo que se "sospecha" que ocurre.





Ejemplo:

Diseñe un contraste que permita evaluar la hipótesis de que "la media de la población X es distinta a la media de la población Y". Diseñe un contraste de manera que la probabilidad de aceptar incorrectamente esta conjetura sea menor al 5%.

En este caso, se tiene que:
* $H_0$: $μ_X = \mu_Y$ (Corresponde a lo que se cree "normal")
* $H_1$: $μ_X \neq \mu_Y$

Además, se plantean de esta manera las hipótesis ya que "aceptar incorrectamente la conjetura" corresponde a un error tipo 1 (valor de $α$) y se sabe que siempre se debe acotar este tipo de error, pues es más grave que un error tipo 2.

# Respuestas

# Disclaimer 

Los datos aquí utilizados fueron obtenidos usando la metodología detallada en [1] y por lo tanto no necesariamente reflejan la opinión de los profesores o ayudantes de este curso. Cualquier alusión a personas o países es meramente circustancial y tiene sólo fines pedagógicos. Todo público puede acceder a los datos en [2] 

[1] Hawkins, K. A., Aguilar, R., Silva, B. C., Jenne, E. K., Kocijan, B., & Kaltwasser, C. R. (2019, June). Measuring populist discourse: The global populism database. In EPSA Annual Conference in Belfast, UK, June (pp. 20-22).

[2] Hawkins, Kirk A.; Aguilar, Rosario; Castanho Silva, Bruno; Jenne, Erin K.; Kocijan, Bojana; Rovira Kaltwasser, Cristóbal, 2019, "Global Populism Database", https://doi.org/10.7910/DVN/LFTQEZ, Harvard Dataverse, V2